In this notebook, we will see how we can use UpTrain package to identify concept drift ie degradation in model's performance. We will use DDM (Drift Detection Method) for the same.

In [1]:
import sys
import os
import subprocess
import zipfile
import numpy as np
import uptrain
import sys
sys.path.insert(0,'..')

from helper_files import read_json, write_json, KpsDataset
import torch

Let's first download the dataset from remote and define the testing and annotation files

In [2]:
data_dir = "data"
remote_url = "https://oodles-dev-training-data.s3.amazonaws.com/data.zip"
orig_training_file = 'data/training_data.json'
if not os.path.exists(data_dir):
    try:
        # Most Linux distributions have Wget installed by default.
        # Below command is to install wget for MacOS
        wget_installed_ok = subprocess.call("brew install wget", shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)
        print("Successfully installed wget")
    except:
        dummy = 1
    try:
        if not os.path.exists("data.zip"):
            file_downloaded_ok = subprocess.call("wget " + remote_url, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)
            print("Data downloaded")
        with zipfile.ZipFile("data.zip", 'r') as zip_ref:
            zip_ref.extractall("./")
        full_training_data = read_json(orig_training_file)
        np.random.seed(1)
        np.random.shuffle(full_training_data)
        reduced_training_data = full_training_data[0:1000]
        write_json(orig_training_file, reduced_training_data)
        print("Prepared Example Dataset")
        os.remove("data.zip")
    except Exception as e:
        print(e)
        print("Could not load training data")

real_world_test_cases = 'data/real_world_testing_data.json'
golden_testing_file = 'data/golden_testing_data.json'
annotation_args = {'master_file': 'data/master_annotation_data.json'}

inference_batch_size = 16

Data downloaded
Prepared Example Dataset


Next, we train our network using Deep Neural Network

In [3]:
from helper_files import get_accuracy_torch, train_model_torch, BinaryClassification
train_model_torch('data/training_data.json', 'version_0')

Training on:  data/training_data.json  which has  1000  data-points
Epoch 0: Loss 6.170424439734028
Epoch 1: Loss 1.804898705902639
Epoch 2: Loss 1.1475218022891212
Epoch 3: Loss 1.1139714462892492
Epoch 4: Loss 0.7651904336334047
Epoch 5: Loss 0.3935121600208127
Epoch 6: Loss 0.40462788252539983
Epoch 7: Loss 0.38329376321783504
Epoch 8: Loss 0.44259468588855605
Epoch 9: Loss 0.3619024118390551
Model saved at:  trained_models_torch/version_0


Next, we get the model accuracy on testing dataset

In [4]:
get_accuracy_torch(golden_testing_file, 'version_0')

Evaluating on  15731  data-points


0.9092873943169538

Let's define the UpTrain config to detect Concept drift and use DDM as the algorithm for the same

In [5]:
cfg = {
    "checks": [{
        'type': uptrain.Anomaly.CONCEPT_DRIFT,
        'algorithm': uptrain.DataDriftAlgo.DDM
    }],
    "data_identifier": "id",
    "tb_logging": True,
    "retraining_folder": "uptrain_smart_data_data_integrity",
}

In [6]:
framework_torch = uptrain.Framework(cfg)

model_dir = 'trained_models_torch/'
model_save_name = 'version_0'
real_world_dataset = KpsDataset(
    real_world_test_cases, batch_size=inference_batch_size, is_test=True
)
model = BinaryClassification()
model.load_state_dict(torch.load(model_dir + model_save_name))
model.eval()

gt_data = read_json(annotation_args['master_file'])
all_gt_ids = [x['id'] for x in gt_data]

for i,elem in enumerate(real_world_dataset):

    # Do model prediction
    inputs = {"data": {"kps": elem[0]["kps"]}, "id": elem[0]["id"]}
    x_test = torch.tensor(inputs["data"]["kps"]).type(torch.float)
    test_logits = model(x_test).squeeze() 
    preds = torch.round(torch.sigmoid(test_logits)).detach().numpy()
    idens = framework_torch.log(inputs=inputs, outputs=preds)

    # Attach ground truth for accuracy evaluation
    this_elem_gt = [gt_data[all_gt_ids.index(x)]['gt'] for x in elem[0]['id']]
    framework_torch.log(identifiers=idens, gts=this_elem_gt)


NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.9.0 at http://localhost:6009/ (Press CTRL+C to quit)


Drift detected with DDM at time:  11030
